In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import urllib
import os
%matplotlib inline
RTSP = os.environ.get('RTSP_ADDRESS')
cap = cv2.VideoCapture(RTSP)
# cap = cv2.VideoCapture(0)

def empty(a):
    pass

cv2.namedWindow("Parameters")
cv2.resizeWindow("Parameters",640,240)
cv2.createTrackbar("Area","Parameters",45,100,empty)
cv2.createTrackbar("lower","Parameters",40,255,empty)
cv2.createTrackbar("upper","Parameters",220,255,empty)
cv2.createTrackbar("Blur_kernel","Parameters",3,5,empty)
cv2.createTrackbar("Dilation_kernel","Parameters",2,5,empty)

def getContours(img,imgContour):
    contours,hierarchy = cv2.findContours(img,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)[-2:]
    for i in range(len(contours)):
        #Filter using area function
        area = cv2.contourArea(contours[i])
        areaMax = cv2.getTrackbarPos("Area","Parameters")
        # If third column value is NOT equal to -1 than its internal
        if hierarchy[0][i][3] != -1 and area < areaMax:
            # Draw the Contour
            cv2.drawContours(imgContour, contours, i, (255,0,255), 2)
            peri = cv2.arcLength(contours[i],True)
            approx = cv2.approxPolyDP(contours[i],0.02*peri,True)
            x,y,w,h = cv2.boundingRect(approx)
            cv2.rectangle(imgContour,(x,y),(x+w,y+h),(0,255,0),1)

while(cap.isOpened()):
    ret, frame = cap.read()
    imgContour = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
#     kernel = np.ones((5,5),np.float32)/25
#     img_blur = cv2.filter2D(gray,-1,kernel)

#     img_blur = cv2.bilateralFilter(gray,25,70,70)
    
#     img_blur = cv2.blur(gray,ksize=(3,3))
    size = cv2.getTrackbarPos("Blur_kernel","Parameters")
#     img_blur = cv2.blur(gray,ksize=(size,size))
    img_blur = cv2.GaussianBlur(gray,(size,size),1)
        
    
    #Cany
    med_val = np.median(img_blur) 
#     lower = int(max(0, 0.7* med_val))
#     upper = int(min(255,1.3 * med_val))
    lower = cv2.getTrackbarPos("lower","Parameters")
    upper = cv2.getTrackbarPos("upper","Parameters")
    edges = cv2.Canny(image=img_blur, threshold1=lower , threshold2=upper)

    #Dilation process
    dsize = cv2.getTrackbarPos("Dilation_kernel","Parameters")
    imgDil = cv2.dilate(edges,np.ones((dsize,dsize)),iterations=1)
    
    #Color inverse
    ret,thresh1 = cv2.threshold(imgDil,100,255,cv2.THRESH_BINARY_INV)
    getContours(thresh1,imgContour)
    
    #Output images
    cv2.imshow('original',frame)
    cv2.imshow('Thresholding after Blur',imgContour[100:600,180:480])
#     cv2.imshow('Thresholding after Blur',thresh1[100:600,180:480])
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
